drop database if exists rnfd_data

In [0]:
%sql
create database if not exists rfnd_data

In [0]:
%sql
drop table if exists rfnd_data.mview_weekly_sales

In [0]:
from pyspark.sql.functions import sum,col,round as spark_round

#----sales Fact table(processed)--------
fact_sales = spark.read.table("hive_metastore.processed_data.fact_sales_performance")
fact_sales=fact_sales.withColumnRenamed("dept_label","prod_type")


#-------Aggregate of the weekly sales data
mview_weekly_sales = (fact_sales.groupBy("store_id", "sku_id", "fsclwk_id", "price_substate_id","prod_type",)
                      .agg(sum("sales_units").alias("sales_units"),
                           sum("sales_dollars").alias("sales_dollars"),
                           sum("discount_dollars").alias("discount_dollars")
                           )
                      #roundin-off columns to 2 decimal places
                      .withColumn("sales_units", spark_round(col("sales_units"), 2))
                      .withColumn("sales_dollars", spark_round(col("sales_dollars"), 2))
                      .withColumn("discount_dollars", spark_round(col("discount_dollars"), 2))
)


mview_weekly_sales.write.mode("overwrite").saveAsTable("rfnd_data.mview_weekly_sales")


In [0]:
%sql
use catalog `hive_metastore`; select * from `rfnd_data`.`mview_weekly_sales` limit 100;

In [0]:
%sql
use catalog `hive_metastore`; select * from `processed_data`.`fact_sales_performance` limit 100;